In [1]:

import numpy 
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


Using TensorFlow backend.


In [2]:
dataframe = pandas.read_csv('neighbors_2017_combined_train_and_test_with_predictions_fixed_with_zeros.csv', usecols=[4,3,10,11,13])

print(dataframe)

         OLS_fitted_value    PU_DO  hour  month   time_delta
0              407.939954  100_164     0      1   554.500000
1              296.614283  100_186     0      1   622.000000
2              376.436955  100_230     0      1   773.000000
3              386.823059   100_48     0      1  1035.500000
4              423.522370   100_68     0      1   631.600000
5              260.728148  107_137     0      1   413.625000
6              362.397935  107_170     0      1   427.577778
7              257.602779  107_224     0      1   657.750000
8              324.761492  107_234     0      1   414.900000
9              295.705884   107_79     0      1   448.619048
10             246.351286  113_114     0      1   300.750000
11             320.900264  113_234     0      1   542.833333
12             291.541252  113_249     0      1   357.800000
13             338.042386   113_79     0      1   426.071429
14             249.854418  114_113     0      1   297.916667
15             296.63505

In [ ]:

import numpy 
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

# fix random seed for reproducibility
numpy.random.seed(7)

Total_dataframe = pandas.read_csv('neighbors_2017_combined_train_and_test_with_predictions_fixed_with_zeros.csv', usecols=[4,3,10,11,13])

PUL_list = numpy.unique(Total_dataframe['PU_DO'])
RMSE_list = [0] * len(PUL_list)
MAPE_list = [0] * len(PUL_list)
Total_dataframe = dataframe

Total_dataframe = pandas.read_csv('neighbors_2017_combined_train_and_test_with_predictions_fixed_with_zeros.csv', usecols=[4,3,10,11,13])
Total_dataframe['Residuals'] = Total_dataframe['time_delta'] - Total_dataframe['OLS_fitted_value']
print(Total_dataframe)

#delete warning flag
pandas.options.mode.chained_assignment = None  # default='warn'

for i in range(len(PUL_list)):    
    print(i/len(PUL_list))
    PULoc = PUL_list[i]
    
    #Training/testing Split
    dataframe = Total_dataframe
    dataframe = dataframe[dataframe['PU_DO'] == PULoc]
    dataframe = dataframe['Residuals']
    dataset = dataframe.values
    dataset = dataset.astype('float32')
    dataset = numpy.reshape(dataset, (len(dataset),1))


    #Scalar transform
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)


    dataframe = Total_dataframe
    dataframe = dataframe[dataframe['PU_DO'] == PULoc]
    dataframe['Residuals'] = dataset
    dataframe = dataframe[dataframe['month'] <= 10]
    dataframe2 = dataframe[dataframe['month'] >= 10]
    dataframe['Residuals']
    dataframe2 = dataframe2['Residuals']
    dataset2 = dataframe2.values
    dataset2 = dataset2.astype('float32')
    dataset2 = numpy.reshape(dataset2, (len(dataset2),1))
    

    

    #Training/Testing Data (Train on Jan-Oct, Ttest on Nov-Dec) 370472 is index
    train_size = int(len(dataset) * 0.9)
    test_size = len(dataset) - train_size
    test2_size = len(dataset2)
    train = dataset[0:train_size,:]
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
    test2 = dataset2

    def create_dataset(dataset, look_back=1):
        dataX, dataY = [], []
        for i in range(len(dataset)-look_back-1):
            a = dataset[i:(i+look_back), 0]
            dataX.append(a)
            dataY.append(dataset[i + look_back, 0])
        return numpy.array(dataX), numpy.array(dataY)

    # reshape into X=t and Y=t+1
    look_back = 1
    trainX, trainY = create_dataset(train, look_back)
    testX, testY = create_dataset(test, look_back)
    test2X, test2Y = create_dataset(test2, look_back)

    # reshape input to be [samples, time steps, features]
    trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    test2X = numpy.reshape(test2X, (test2X.shape[0], 1, test2X.shape[1]))



    # create and fit the LSTM network
    model = Sequential()
    #model.add(LSTM(1, input_shape=(1, look_back)))
    model.add(LSTM(4))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainX, trainY, epochs=10, batch_size=10, verbose=0)




    # make predictions
    trainPredict = model.predict(trainX)
    testPredict = model.predict(testX)
    test2Predict = model.predict(test2X)
     #invert predictions
    trainPredict = scaler.inverse_transform(trainPredict)
    trainY = scaler.inverse_transform([trainY])
    testPredict = scaler.inverse_transform(testPredict)
    testY = scaler.inverse_transform([testY])
    test2Predict = scaler.inverse_transform(test2Predict)
    test2Y = scaler.inverse_transform([test2Y])
    # calculate root mean squared error
    
    trainScore = math.sqrt(mean_squared_error(trainY[0] ,  trainPredict[:,0]))
    print('Train Score: %.2f RMSE' % (trainScore))
    testScore = math.sqrt(mean_squared_error(testY[0], testPredict[:,0]))
    print('Test Score: %.2f RMSE' % (testScore))
    test2Score = math.sqrt(mean_squared_error(test2Y[0], test2Predict[:,0]))
    print('Test2 Score: %.2f RMSE' % (test2Score))

           #
    #print('training actual:', trainY[0])
    #print('training predict:', trainPredict[:,0]  )
    #print(numpy.mean(abs(trainY[0]   -trainPredict[:,0]  )))

    #print('testing actual:', testY[0] )
    #print('testing predict:', testPredict[:,0] )
    #print(numpy.mean( abs(testY[0] - testPredict[:,0]  )))

    # shift train predictions for plotting
    #trainPredictPlot = numpy.empty_like(dataset)
    #trainPredictPlot[:, :] = numpy.nan
    #trainPredictPlot[look_back:len(trainPredict)+look_back, :] = trainPredict
    # shift test predictions for plotting
    #testPredictPlot = numpy.empty_like(dataset)
    #testPredictPlot[:, :] = numpy.nan
    #testPredictPlot[len(trainPredict)+(look_back*2)+1:len(dataset)-1, :] = testPredict
    # plot baseline and predictions
    #plt.plot(scaler.inverse_transform(dataset))
    #plt.plot(trainPredictPlot)
    #plt.plot(testPredictPlot)

    print(test2Y[0][0:10])
    print(test2Predict[:,0][0:10])



    RMSE_list[i] = test2Score
    Test2_MAPE = numpy.mean(abs(test2Y[0]- test2Predict[:,0] ))  
    MAPE_list[i] = Test2_MAPE
    print('Test_MAD:', Test2_MAPE)
    
    #print('Testy:', (testY[0])[0:10])
    #print('Testpredict:', testPredict[:,0])
    
    Test_MAPE = numpy.mean(abs(testY[0]- testPredict[:,0] )  )  
    print('Validation_MAD:', Test_MAPE)

    Test2_MAD = numpy.mean(abs(test2Y[0]- test2Predict[:,0]))
    Test_MAD = numpy.mean(abs(testY[0]- testPredict[:,0]))
    #print('Test_MAD:', Test2_MAPE)
    #print('Validation_MAD:', Test_MAPE)
    
        #Save the Model
    filename = 'Time_LSTM_Resid' + str(i)  +'.sav'
    pickle.dump(model, open(filename, 'wb'))
    

         OLS_fitted_value    PU_DO  hour  month   time_delta   Residuals
0              407.939954  100_164     0      1   554.500000  146.560046
1              296.614283  100_186     0      1   622.000000  325.385717
2              376.436955  100_230     0      1   773.000000  396.563045
3              386.823059   100_48     0      1  1035.500000  648.676941
4              423.522370   100_68     0      1   631.600000  208.077630
5              260.728148  107_137     0      1   413.625000  152.896852
6              362.397935  107_170     0      1   427.577778   65.179842
7              257.602779  107_224     0      1   657.750000  400.147221
8              324.761492  107_234     0      1   414.900000   90.138508
9              295.705884   107_79     0      1   448.619048  152.913163
10             246.351286  113_114     0      1   300.750000   54.398714
11             320.900264  113_234     0      1   542.833333  221.933069
12             291.541252  113_249     0      1   3

0.039473684210526314
Train Score: 86.28 RMSE
Test Score: 113.54 RMSE
Test2 Score: 105.41 RMSE
[ 120.79078977   81.23751479   45.08451418   63.36679049  154.89041941
   47.64252357 -187.58474554  -10.25696321   57.34670023   16.85450367]
[  61.0677     88.070526   58.379887   31.64807    45.11599   114.015656
   33.52611  -129.43073    -8.461168   40.66966 ]
Test_MAD: 83.49024231480394
Validation_MAD: 93.69224215847059
0.043859649122807015
Train Score: 103.67 RMSE
Test Score: 158.63 RMSE
Test2 Score: 141.38 RMSE
[   5.50254342  187.29797361   83.08198376 -158.5786105    64.0783243
 -106.00288552  135.76988176  -17.20235534   21.86799544   14.10911807]
[ 41.57846     2.1326737 103.13157    44.326813  -82.00707    33.859344
 -55.85058    73.769966   -9.93802    10.912967 ]
Test_MAD: 113.16452918614061
Validation_MAD: 132.47877695422272
0.04824561403508772
Train Score: 110.24 RMSE
Test Score: 141.71 RMSE
Test2 Score: 132.10 RMSE
[  62.4535433    74.08234914  -25.71701581 -159.12759435 -175

0.12719298245614036
Train Score: 102.36 RMSE
Test Score: 150.16 RMSE
Test2 Score: 133.59 RMSE
[-59.64594545 -48.88939416  45.53347618 -15.21043796 204.27983142
  80.19862087  -5.92863577  49.57058313   4.21233947 -34.08936485]
[-55.71689  -40.473007 -34.201588  23.6182   -14.138674 130.75937
  46.021484  -8.497882  26.196257  -2.281033]
Test_MAD: 103.56281937095801
Validation_MAD: 120.4840308775476
0.13157894736842105
Train Score: 166.46 RMSE
Test Score: 252.01 RMSE
Test2 Score: 228.67 RMSE
[418.10952141 141.48836407 -22.22768387 -75.38827099 136.76866782
  44.18748362  37.96019926  87.4879954   35.89166364   6.79945971]
[ 87.26561  157.11136   55.07877   -0.586766 -17.88979   53.424274
  21.564379  19.462383  36.32371   18.765257]
Test_MAD: 182.00461184516172
Validation_MAD: 212.02304209578452
0.13596491228070176
Train Score: 112.50 RMSE
Test Score: 149.63 RMSE
Test2 Score: 134.79 RMSE
[ -62.88666871   -8.84120493   33.64275221  -21.3844722   -73.56087365
   24.35789644  146.13064109 

0.2149122807017544
Train Score: 84.16 RMSE
Test Score: 122.22 RMSE
Test2 Score: 110.58 RMSE
[ 14.45638472  13.6684775  -58.04754122 124.79188338  87.94883674
 -89.63239795  67.71177324  72.04313544  34.15781555  78.89626954]
[-16.059532  12.572403  12.080258 -31.837473  83.418205  59.34647
 -50.613213  46.297752  49.08002   24.94316 ]
Test_MAD: 87.3196731042769
Validation_MAD: 101.51082213231642
0.21929824561403508
Train Score: 153.20 RMSE
Test Score: 224.29 RMSE
Test2 Score: 207.76 RMSE
[  44.38293982 -273.5716311  -253.28765298 -224.44823728   54.04206422
  128.12752248 -204.09976411   -5.90532359  -15.83497714    3.57283764]
[  85.56515     6.719065 -136.85114  -128.4008   -116.213036   11.432294
   48.21446  -107.49236   -17.501993  -22.220608]
Test_MAD: 165.4408417674733
Validation_MAD: 187.53110711339394
0.2236842105263158
Train Score: 154.73 RMSE
Test Score: 217.65 RMSE
Test2 Score: 201.52 RMSE
[ -12.24436247  101.71778325  372.2517613  -187.90884719 -130.75189691
 -135.33310295

0.3026315789473684
Train Score: 122.10 RMSE
Test Score: 150.16 RMSE
Test2 Score: 150.22 RMSE
[ -39.18184343 -183.38639471   -3.10243301  -39.26298512  -20.10606114
   -5.68727029    6.08545941   27.61323787   78.30857869    6.224717  ]
[-58.95906     2.385726  -98.13395    28.071447    2.3282452  15.9408655
  26.224539   34.64476    50.09657    86.753296 ]
Test_MAD: 109.0705390107991
Validation_MAD: 117.45203877567994
0.30701754385964913
Train Score: 122.94 RMSE
Test Score: 186.26 RMSE
Test2 Score: 165.91 RMSE
[  11.295307     21.59078004  -10.04190545 -301.53581841 -141.37888256
 -248.96013081  203.81258778  159.34041227   16.91075126  -55.24809241]
[   4.13337    26.138186   32.801586   12.430715 -159.77582   -68.761986
 -130.91669   155.69673   124.88146    29.76859 ]
Test_MAD: 124.849282772453
Validation_MAD: 148.57383477462778
0.31140350877192985
Train Score: 124.94 RMSE
Test Score: 214.75 RMSE
Test2 Score: 183.47 RMSE
[-85.35492883 -17.05945502 -19.97547138 110.56393861  85.22088

0.39035087719298245
Train Score: 97.09 RMSE
Test Score: 144.75 RMSE
Test2 Score: 135.39 RMSE
[ 51.51645258  37.8476131    8.17920532  -2.56471304  92.84223384
 189.26102005 221.53372042   7.72820327  59.63188939  33.03968225]
[118.03395   39.404453  30.63665   11.651701   4.793591  65.98116
 128.26013  149.14658   11.363613  44.61577 ]
Test_MAD: 100.53532948338233
Validation_MAD: 115.05323487135517
0.39473684210526316
Train Score: 86.89 RMSE
Test Score: 131.37 RMSE
Test2 Score: 115.71 RMSE
[   0.98085909  -36.64035346   78.8436268    30.14972844  -56.35999878
  336.05874544  125.16484169   91.35931322  -39.57035333 -132.82921995]
[-10.867625   -2.5747662 -27.503942   50.882698   17.168167  -40.319553
 241.32727    83.765945   59.692764  -29.419285 ]
Test_MAD: 90.58647915830804
Validation_MAD: 107.59722284995448
0.3991228070175439
Train Score: 104.35 RMSE
Test Score: 166.75 RMSE
Test2 Score: 149.45 RMSE
[ 136.39629789   62.70367446   45.65429638   63.32707461   50.15066405
    8.2360945

In [17]:
import numpy
numpy.savetxt("LSTM_Time_Resid_MAD.csv", MAD_list, delimiter=",")
numpy.savetxt("LSTM_Time_Resid_RMSE.csv", RMSE_list, delimiter=",")

NameError: name 'MAD_list' is not defined

In [6]:
print(type(model))

import pickle
a = [model,model]


<class 'keras.engine.sequential.Sequential'>


In [7]:
loaded_model = pickle.load(open(filename, 'rb'))

In [8]:
print(loaded_model)

In [9]:
'a' + 'b'

'ab'